In [1]:
import glob
from astropy.io import fits
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
from matplotlib import pyplot as plt
from astropy.stats import sigma_clip
import numpy as np
import os
from astropy.wcs import WCS
from reproject import reproject_exact
from astropy.stats import sigma_clipped_stats
#from photutils.profiles import RadialProfile
#from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus, ApertureStats
from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
#from photutils.centroids import centroid_1dg

#code taken from assignment 2 (ccd reduction)-- we used Dylan's code for reduction since we did it collectively as a group.
def create_median_bias(bias_list, median_bias_filename):
    # Creates a list of 2d numpy arrays from all of the files in bias_list
    bias_images = []
    for img_file in bias_list:
        bias_data = fits.getdata(img_file)
        bias_images.append(bias_data.astype('f4'))

    bias_stack = np.stack(bias_images, axis=0)

    # Sigma clips the data inside each 2d numpy array
    bias_images_masked = sigma_clip(bias_stack, cenfunc='median', sigma=3.0, axis=0)

    '''
    # Sigma clips the data inside each 2d numpy array
    bias_images_masked = sigma_clip(bias_images, cenfunc='median', sigma=3.0, axis=0)
    '''
    median_bias = (np.ma.median(bias_images_masked, axis=0)).data

    # Creates a new file of the median bias
    primary = fits.PrimaryHDU(data=median_bias, header=fits.Header())
    hdul = fits.HDUList([primary])
    hdul.writeto(median_bias_filename, overwrite=True)

    # Returns the new median bias frame as a 2d numpy array
    return median_bias

from astropy.io import fits
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
from matplotlib import pyplot as plt
from astropy.stats import sigma_clip
import numpy as np

def create_median_dark(dark_list, bias_filename, median_dark_filename):
    # Initialize lists of regular dark data and dark data without bias
    dark_images = []
    bias_data = fits.getdata(bias_filename)

    for file in dark_list:
        # Open data in dark files
        dark = fits.open(file)
        dark_data = dark[0].data.astype('f4')
        dark_header = dark[0].header
        exptime = dark_header['EXPTIME']

        # Subtract bias
        dark_data_no_bias = dark_data - bias_data

        # Update dark_images with dark current
        dark_images.append(dark_data_no_bias / exptime)

    # Sigma-clip the dark frames by sigma 3
    dark_sc = sigma_clip(dark_images, cenfunc='median', sigma=3, axis=0)

    # Combine dark frames into single dark
    median_dark = np.ma.mean(dark_sc, axis=0)

    # Save combined dark frame to disk
    # Set the EXPTIME to 1 second.
    dark_hdu = fits.PrimaryHDU(data=median_dark.data, header=dark[0].header)
    dark_hdu.header['EXPTIME'] = 1
    dark_hdu.header['COMMENT'] = 'Combined dark image with bias subtracted'
    dark_hdu.header['BIASFILE'] = (str(bias_filename), 'Bias image used to subtract bias level')

    # Save the dark image to disk
    dark_hdu.writeto(median_dark_filename, overwrite=True)
    # See code in create_median_bias for how to create a new FITS file
    # from the resulting median dark frame.

    return median_dark

from astropy.io import fits
from astropy.visualization import ImageNormalize, LinearStretch, ZScaleInterval
from matplotlib import pyplot as plt
from astropy.stats import sigma_clip
import numpy as np

def create_median_flat(
    flat_list,
    bias_filename,
    median_flat_filename,
    dark_filename=None,
):
    # Initialize lists for 2d flat numpy arrays and for filter and exposure time of each flat
    filter_type = []
    flat_images = []
    flat_exptime = []
    for file in flat_list:
          flat = fits.open(file)
          filter_type.append(flat[0].header["FILTER"])
          flat_exptime.append(flat[0].header["EXPTIME"])
          flat_data = fits.getdata(file)
          flat_images.append(flat_data.astype('f4'))

    # Creates a boolean to check if all filters are the same
    same_filter = all(elem == filter_type[0] for elem in filter_type)

    # If the filters are not the same then a string detailing the issue is returned
    if (not same_filter):
          return 'Error, not all flats are from the same filter'

    # Subtracts bias data from flat data
    bias_data = fits.getdata(bias_filename)
    flat_no_bias = []
    for data in flat_images:
        flat_no_bias_data =  data - bias_data
        flat_no_bias.append(flat_no_bias_data)

    # Subtracts dark data if it exists
    if dark_filename:
        dark_data = fits.getdata(dark_filename)
        flat_no_bias_dark = []
        count = 0
        # Insures dark data is scaled by exposure time of flat
        for data in flat_no_bias:
            flat_no_bias_dark_data = data - (dark_data * flat_exptime[count])
            flat_no_bias_dark.append(flat_no_bias_dark_data)
            count += 1

    # Sigma clip the bias-subtracted and or possibly dark-subtracted flats at sigma = 3
    if dark_filename:
        flat_sc = sigma_clip(flat_no_bias_dark, cenfunc='median', sigma=3, axis=0)
    else:
        flat_sc = sigma_clip(flat_no_bias, cenfunc='median', sigma=3, axis=0)

    # Combine flat frames into a single median flat
    median_flat = np.ma.mean(flat_sc, axis=0)
    normalized_flat = median_flat / np.median(median_flat)

    # Saves median flat frame to input name from 'median_flat_filename'
    primary = fits.PrimaryHDU(data=normalized_flat.data, header=fits.Header())
    hdul = fits.HDUList([primary])
    hdul.writeto(median_flat_filename, overwrite=True)

    # See code in create_median_bias for how to create a new FITS file
    # from the resulting median flat frame.

    return normalized_flat

from astropy.io import fits

def reduce_science_frame(
    science_filename,
    median_bias_filename,
    median_flat_filename,
    median_dark_filename,
    reduced_science_filename="reduced_science.fits",
):
    # Get data from each file type
    sci_data = fits.getdata(science_filename)
    bias_data = fits.getdata(median_bias_filename)
    dark_data = fits.getdata(median_dark_filename)
    flat_data = fits.getdata(median_flat_filename)

    # Get the original header to be saved to the file
    sci_header = fits.getheader(science_filename)
    
    # Get science data exposure time for dark current reduction
    sci_exposure = fits.getheader(science_filename)['EXPTIME']

    # Reduce science image
    reduced_science = (sci_data - bias_data - (dark_data * sci_exposure)) / flat_data

    # Save reduced science image
    hdu = fits.PrimaryHDU(reduced_science, header=sci_header)
    hdu.header['BUNIT'] = 'ADU'
    hdu.header['COMMENT'] = 'Reduced science image: bias, dark, and flat corrected'
    hdu.writeto(reduced_science_filename, overwrite=True)

    # Return fully cleaned/reduced science image
    return reduced_science

def do_aperture_photometry(
    image,
    positions,
    radii,
    sky_radius_in,
    sky_annulus_width,
):
    # Open up the reduced science image and get its data
    image_data = fits.getdata(image)

    # Set an index counter radii based off how many different coords 
    # are inside of positions
    radii_count = 0

    # Initialize astropy Qtable for storing output photometry info
    phot_table = []

    # Loop through every different object to do photometry of
    for coords in positions:

        # Create coords of the object
        x_coord = coords[0]
        y_coord = coords[1]

        # Find aperture, annulus, and media background value
        aperture = CircularAperture((x_coord, y_coord), r=radii[radii_count])
        annulus = CircularAnnulus((x_coord, y_coord), r_in=sky_radius_in, r_out= sky_radius_in + sky_annulus_width)
        back = (ApertureStats(image_data, annulus)).median

        # Create table and prepare data
        phot = aperture_photometry(image_data, aperture)
        flux_raw = phot['aperture_sum'][0]
        aperture_area = aperture.area_overlap(image_data)
        flux_no_back = flux_raw - back * aperture_area

        # Adds more info to table
        phot['aperture_sum_no_back'] = flux_no_back
        phot['id'] = radii_count
        phot['radius'] = radii[radii_count]
        phot['xcenter'] = x_coord
        phot['ycenter'] = y_coord
        phot['aperture_radius'] = sky_radius_in
        
        phot_table.append(phot)

        # Update radii count
        radii_count += 1
        
    # Creates a final table out of each individual table
    final_phot_table = np.vstack(phot_table)

    # Returns the table summing all objects to do aperture phot on
    return final_phot_table

def background_from_photometry(aperture_data, radii):
    backgrounds = []
    for row, r in zip(aperture_data, radii):
        row_data = row[0]  # Unpack the single-element array
        flux_raw = row_data[3]  # aperture_sum (Jy)
        flux_no_back = row_data[4]  # aperture_sum_no_back (Jy)
        aperture_area = np.pi * r**2  # Approximate area (pixels)
        back = (flux_raw - flux_no_back) / aperture_area  # Jy/pixel
        backgrounds.append(back)
    mean_back = np.mean(backgrounds)
    print(f"Backgrounds: {backgrounds}, Mean: {mean_back:.2e} Jy/pixel")
    return mean_back
  
def subtract_background(image_file, background, output_file):
    with fits.open(image_file) as hdul:
        image_data = hdul[0].data
        header = hdul[0].header
        new_data = image_data - background
        header['COMMENT'] = f'Subtracted background: {background:.2e} Jy/pixel'
        hdu = fits.PrimaryHDU(new_data, header=header)
        hdu.writeto(output_file, overwrite=True)
        print(f"Saved: {output_file}, Sample: {new_data[500:505, 500]}")

def compute_error_image(adu_files, science_file, zmag, exptime, gain=1.25, rdnoise=11.8):
    error_squares = []
    with fits.open(science_file) as hdul:
        science_data = hdul[0].data.astype(np.float32, copy=False)
    
    for file in adu_files:
        with fits.open(file) as hdul:
            counts = hdul[0].data.astype(np.float32, copy=False)
            counts_aligned, _ = aa.register(counts, science_data, max_control_points=100, detection_sigma=3)
            noise_electrons = np.sqrt(np.maximum(counts_aligned / gain, 0) + rdnoise**2)
            noise_counts = noise_electrons * gain
            count_rate = counts_aligned / exptime
            mask = count_rate > 0
            flux = np.zeros_like(counts_aligned)
            flux[mask] = 3631 * 10**(-0.4 * (zmag - 2.5 * np.log10(count_rate[mask])))
            error_jy = np.zeros_like(counts_aligned)
            error_jy[mask] = flux[mask] * (noise_counts[mask] / np.maximum(counts_aligned[mask], 1e-8))
            error_squares.append(error_jy**2)
    
    combined_error = np.sqrt(np.nanmean(error_squares, axis=0))
    return combined_error

def create_error_image(science_file, adu_files, zmag, exptime, output_dir, gain=1.25, rdnoise=11.8):
    error_data = compute_error_image(adu_files, science_file, zmag, exptime, gain, rdnoise)
    
    with fits.open(science_file) as hdul:
        header = hdul[0].header
    error_file = os.path.join(output_dir, f'error_{os.path.basename(science_file)}')
    hdu = fits.PrimaryHDU(error_data, header=header)
    hdu.header['BUNIT'] = 'Jy/pixel'
    hdu.header['COMMENT'] = f'Noise image rdnoise={rdnoise} e-'
    hdu.writeto(error_file, overwrite=True)
    print(f"Saved: {error_file}, Sample: {error_data[500:505, 500]}")

In [68]:
# Creates a list out of all bias frames available in directory
bias_list = glob.glob('Bias_BIN1_*.fits')
# Creates a median bias from the bias frames in 'bias_list'
create_median_bias(bias_list, 'median_bias.fit')

array([[1376., 1391., 1389., ..., 1390., 1381., 1379.],
       [1316., 1308., 1324., ..., 1314., 1309., 1314.],
       [1319., 1325., 1328., ..., 1323., 1323., 1331.],
       ...,
       [1374., 1379., 1374., ..., 1375., 1380., 1380.],
       [1369., 1381., 1374., ..., 1390., 1367., 1387.],
       [1371., 1384., 1384., ..., 1384., 1373., 1374.]], dtype=float32)

In [69]:
# Creates a list out of all dark frames present in the directory
dark_list = glob.glob('Dark_BIN1_*.fits')
# Creates a median bias using all dark frames inside 'dark_list' and the generated median bias frame
# from the previous cell
create_median_dark(dark_list, 'median_bias.fit', 'median_dark.fit')

masked_array(
  data=[[0.07799999713897705, 0.029333335161209107, 0.05400000214576721,
         ..., 0.04733333289623261, 0.0753333330154419,
         0.051999998092651364],
        [0.04866666793823242, 0.08533333539962769, 0.04866666793823242,
         ..., 0.2693333148956299, 0.0753333330154419,
         0.08733333349227905],
        [0.08399999737739564, 0.04333333671092987, 0.04466666579246521,
         ..., 0.046666663885116574, 0.06399999856948853,
         0.01133333370089531],
        ...,
        [0.09933333396911621, 0.058666670322418214, 0.07200000286102295,
         ..., 0.05533333420753479, 0.027333331108093262,
         0.07333334088325501],
        [0.10866667032241821, 0.42800002098083495, 0.11333333253860474,
         ..., 0.03999999761581421, 0.09533333778381348,
         0.03866666555404663],
        [0.08933333158493043, 0.028666666150093077, 0.04466666579246521,
         ..., 0.40266666412353513, 0.07400000095367432,
         0.07733333110809326]],
  mask=[[False,

In [70]:
# Creates a list of flats for each associated filter present in the directory
flat_list_g = glob.glob('domeflat_g_*.fits')
flat_list_i = glob.glob('domeflat_i_*.fits')
flat_list_r = glob.glob('domeflat_r_*.fits')
flat_list_z = glob.glob('domeflat_z_*.fits')
# Creates a median flat for every filter of 'flat_list' present above. Uses previously generated
# median bias and median dark frame from above cells.
median_flat_g = create_median_flat(flat_list_g, 'median_bias.fit', 'median_flat_g.fit', 'median_dark.fit')
median_flat_i = create_median_flat(flat_list_i, 'median_bias.fit', 'median_flat_i.fit', 'median_dark.fit')
median_flat_r = create_median_flat(flat_list_r, 'median_bias.fit', 'median_flat_r.fit', 'median_dark.fit')
median_flat_z = create_median_flat(flat_list_z, 'median_bias.fit', 'median_flat_z.fit', 'median_dark.fit')

/home/dberry/anaconda3/envs/pixedfit_testing/lib/python3.12/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [85]:
# Creates a list of ALL science images taken
science_g = glob.glob('NGC 6946_g*.fits')
science_i = glob.glob('NGC 6946_i*.fits')
science_r = glob.glob('NGC 6946_r*.fits')
science_z = glob.glob('NGC 6946_z*.fits')
# Reduces each science frame depending on the filter used for the photo
for i in range(len(science_g)):
    reduce_science_frame(science_g[i], 'median_bias.fit', 'median_flat_g.fit', 'median_dark.fit', f'new_reduced_science_g{i}.fits')
for i in range(len(science_i)):
    reduce_science_frame(science_i[i], 'median_bias.fit', 'median_flat_i.fit', 'median_dark.fit', f'new_reduced_science_i{i}.fits')
for i in range(len(science_r)):
    reduce_science_frame(science_r[i], 'median_bias.fit', 'median_flat_r.fit', 'median_dark.fit', f'new_reduced_science_r{i}.fits')
for i in range(len(science_z)):
    reduce_science_frame(science_z[i], 'median_bias.fit', 'median_flat_z.fit', 'median_dark.fit', f'new_reduced_science_z{i}.fits')

In [86]:
def adu_to_jy(list_files, curr_dir = '', reduced = False):
    for file in list_files:
        # Open up fits file
        hdul = fits.open(curr_dir + file)
        
        # Extract data and header
        data = hdul[0].data
        header = hdul[0].header
        
        # Convert raw units into adu counts using BZERO and BSCALE
        if not reduced:
            adu_counts = header['BZERO'] + header['BSCALE'] * data
            print(f'BZERO= {header['BZERO']}, BSCALE= {header['BSCALE']}.')
        else:
            adu_counts = data
            
        # Convert adu counts to a count rate using EXPTIME
        count_rate = adu_counts / header['EXPTIME'] 
        print(f'EXPTIME= {header['EXPTIME']}')
        
        # Convert to a magnitude using ZMAG
        mag = header['ZMAG'] - 2.5 * np.log10(count_rate)
        print(f'ZMAG=  {header['ZMAG']}')
        
        # Convert to flux values in Jy using common zero point flux
        zero_point_flux = 3631
        flux = zero_point_flux * 10**(-0.4 * mag)
        
        # Save flux-calibrated image
        output_filename = f'flux_calibrated_{file}'
        hdu = fits.PrimaryHDU(flux, header=header)
        hdu.header['BUNIT'] = 'Jy/pixel'
        hdu.header['COMMENT'] = 'Flux-calibrated image in Janskys per pixel'
        hdu.writeto(os.path.join(curr_dir, output_filename), overwrite=True)
        print(f'Saved flux-calibrated image as {output_filename}')

        # Close the open fits file just in case
        hdul.close()
        
        # Print that the values have been converted and provide a small selection
        print('Flux now in units of Jy, EXAMPLE:', flux[500:505,500])

science_g = glob.glob(os.path.join('new_reduced_science_g*.fits'))
science_i = glob.glob(os.path.join('new_reduced_science_i*.fits'))
science_r = glob.glob(os.path.join('new_reduced_science_r*.fits'))
science_z = glob.glob(os.path.join('new_reduced_science_z*.fits'))

adu_to_jy(science_g, curr_dir, True)
adu_to_jy(science_r, curr_dir, True)
adu_to_jy(science_i, curr_dir, True)
adu_to_jy(science_z, curr_dir, True)

EXPTIME= 360.0
ZMAG=  20.6181373634
Saved flux-calibrated image as flux_calibrated_new_reduced_science_g0.fits
Flux now in units of Jy, EXAMPLE: [2.11971020e-05 2.01517268e-05 2.18899600e-05 2.08516790e-05
 2.19390882e-05]
EXPTIME= 360.0
ZMAG=  20.5949255573
Saved flux-calibrated image as flux_calibrated_new_reduced_science_g1.fits
Flux now in units of Jy, EXAMPLE: [1.97511223e-05 2.13290460e-05 2.08860515e-05 2.03192601e-05
 1.91673608e-05]


/tmp/ipykernel_499/2425285997.py:22: RuntimeWarning: invalid value encountered in log10
  mag = header['ZMAG'] - 2.5 * np.log10(count_rate)


EXPTIME= 360.0
ZMAG=  20.6281139709
Saved flux-calibrated image as flux_calibrated_new_reduced_science_g2.fits
Flux now in units of Jy, EXAMPLE: [1.98840017e-05 2.26794650e-05 1.87697121e-05 1.77446420e-05
 2.10756611e-05]
EXPTIME= 360.0
ZMAG=  20.5921568932
Saved flux-calibrated image as flux_calibrated_new_reduced_science_g3.fits
Flux now in units of Jy, EXAMPLE: [1.89338753e-05 2.38436153e-05 2.25909299e-05 1.93856276e-05
 2.30983815e-05]
EXPTIME= 360.0
ZMAG=  20.6361974692
Saved flux-calibrated image as flux_calibrated_new_reduced_science_g4.fits
Flux now in units of Jy, EXAMPLE: [1.86255968e-05 1.88853565e-05 2.12007376e-05 1.69450173e-05
 1.75204885e-05]
EXPTIME= 360.0
ZMAG=  21.3949438559
Saved flux-calibrated image as flux_calibrated_new_reduced_science_r0.fits
Flux now in units of Jy, EXAMPLE: [2.50552449e-05 2.78727553e-05 2.57879233e-05 2.43233381e-05
 2.42044564e-05]
EXPTIME= 360.0
ZMAG=  21.3427630018
Saved flux-calibrated image as flux_calibrated_new_reduced_science_r1.fi

In [92]:
import astroalign as aa
from astropy.io import fits
import numpy as np
import os

def align_images(filter_name, num_images):
    import astroalign as aa
    from astropy.io import fits
    import numpy as np
    import os

    aligned_dir = f"aligned_{filter_name}"
    os.makedirs(aligned_dir, exist_ok=True)

    ref_path = f"flux_calibrated_new_reduced_science_{filter_name}0.fits"
    ref_data = fits.getdata(ref_path).astype(np.float32)

    for i in range(num_images):
        input_path = f"flux_calibrated_new_reduced_science_{filter_name}{i}.fits"
        output_path = os.path.join(aligned_dir, f"aligned_science_{filter_name}{i}.fits")

        print(f"Aligning {input_path} to {ref_path}...")

        try:
            data = fits.getdata(input_path).astype(np.float32)
            header = fits.getheader(input_path)

            aligned_data, _ = aa.register(data, ref_data)
            fits.writeto(output_path, aligned_data, header, overwrite=True)

        except aa.MaxIterError:
            print(f"Alignment failed for {input_path}. Skipping.")
        except Exception as e:
            print(f"Unexpected error with {input_path}: {e}")

    print(f"Done aligning filter {filter_name}.")
# Example usage:
align_images('g', 5)
align_images('r', 5)
align_images('i', 5)
align_images('z', 7)

Aligning flux_calibrated_new_reduced_science_g0.fits to flux_calibrated_new_reduced_science_g0.fits...
Aligning flux_calibrated_new_reduced_science_g1.fits to flux_calibrated_new_reduced_science_g0.fits...
Aligning flux_calibrated_new_reduced_science_g2.fits to flux_calibrated_new_reduced_science_g0.fits...
Aligning flux_calibrated_new_reduced_science_g3.fits to flux_calibrated_new_reduced_science_g0.fits...
Aligning flux_calibrated_new_reduced_science_g4.fits to flux_calibrated_new_reduced_science_g0.fits...
Done aligning filter g.
Aligning flux_calibrated_new_reduced_science_r0.fits to flux_calibrated_new_reduced_science_r0.fits...
Aligning flux_calibrated_new_reduced_science_r1.fits to flux_calibrated_new_reduced_science_r0.fits...
Aligning flux_calibrated_new_reduced_science_r2.fits to flux_calibrated_new_reduced_science_r0.fits...
Aligning flux_calibrated_new_reduced_science_r3.fits to flux_calibrated_new_reduced_science_r0.fits...
Aligning flux_calibrated_new_reduced_science_r4.f

In [89]:
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clip
import glob
import os

def sigma_clip_combine(filter_name):
    aligned_dir = f'aligned_{filter_name}'
    output_path = f'combined_{filter_name}.fits'

    file_list = sorted(glob.glob(os.path.join(aligned_dir, f'flux_calibrated_aligned_science_{filter_name}*.fits')))
    image_stack = []

    for file in file_list:
        data = fits.getdata(file).astype(np.float32)
        image_stack.append(data)

    stack = np.stack(image_stack)

    # Apply sigma clipping along the stack axis (axis=0)
    clipped = sigma_clip(stack, sigma=3.0, axis=0)

    # Use mean of non-masked values
    combined = np.ma.mean(clipped, axis=0)

    header = fits.getheader(file_list[0])
    fits.writeto(output_path, combined.filled(np.nan), header, overwrite=True)

    print(f"Saved sigma-clipped combined image: {output_path}")

sigma_clip_combine('g')
sigma_clip_combine('r')
sigma_clip_combine('i')
sigma_clip_combine('z')

Saved sigma-clipped combined image: combined_g.fits
Saved sigma-clipped combined image: combined_r.fits
Saved sigma-clipped combined image: combined_i.fits
Saved sigma-clipped combined image: combined_z.fits


In [94]:
import numpy as np
from astropy.io import fits
import glob
import os

def robust_median_combine(filter_name, zero_threshold=True):
    aligned_dir = f'aligned_{filter_name}'
    output_path = f'combined_{filter_name}_robust.fits'

    file_list = sorted(glob.glob(os.path.join(aligned_dir, f'aligned_science_{filter_name}*.fits')))
    stack = []

    for file in file_list:
        data = fits.getdata(file).astype(np.float32)

        # Optional: treat zeros as bad pixels (can change to any threshold)
        if zero_threshold:
            data[data <= 0] = np.nan  # also catches negative/noisy interpolated areas

        stack.append(data)

    # Use nanmedian to ignore NaNs (e.g., from zero-masking or WCS misalignment)
    combined = np.nanmedian(np.stack(stack), axis=0)

    header = fits.getheader(file_list[0])
    fits.writeto(output_path, combined, header, overwrite=True)

    print(f"Saved robust combined image: {output_path}")
robust_median_combine('g')
robust_median_combine('r')
robust_median_combine('i')
robust_median_combine('z')

/tmp/ipykernel_499/1311142317.py:23: RuntimeWarning: All-NaN slice encountered
  combined = np.nanmedian(np.stack(stack), axis=0)


Saved robust combined image: combined_g_robust.fits
Saved robust combined image: combined_r_robust.fits
Saved robust combined image: combined_i_robust.fits
Saved robust combined image: combined_z_robust.fits


In [95]:
import astroalign as aa
from astropy.io import fits
import numpy as np

def align_to_reference(ref_path, target_path, output_path):
    # Load reference image (g-band)
    ref_data = fits.getdata(ref_path).astype(np.float32)

    # Load target image (to align)
    target_data = fits.getdata(target_path).astype(np.float32)
    target_header = fits.getheader(target_path)

    # Optional: mask non-positive values (can improve robustness)
    target_data[target_data <= 0] = np.nan
    ref_data[ref_data <= 0] = np.nan

    # Align
    aligned_data, _ = aa.register(target_data, ref_data)

    # Save aligned image
    fits.writeto(output_path, aligned_data, target_header, overwrite=True)
    print(f"Aligned {target_path} → {output_path}")
# Align r, i, z to g
align_to_reference("combined_i_robust.fits", "combined_r_robust.fits", "aligned_to_i_r.fits")
align_to_reference("combined_i_robust.fits", "combined_g_robust.fits", "aligned_to_i_g.fits")
align_to_reference("combined_i_robust.fits", "combined_z_robust.fits", "aligned_to_i_z.fits")

Aligned combined_r_robust.fits → aligned_to_i_r.fits
Aligned combined_g_robust.fits → aligned_to_i_g.fits
Aligned combined_z_robust.fits → aligned_to_i_z.fits


In [133]:
positions = [(496, 131), (104, 302), (75,203), 
             (212, 681), (876, 216), (833, 635), (853, 864)]
radii = [10, 11, 8, 8, 12, 8, 10]
sky_radius_in = 10
sky_annulus_width = 5

aligned_reduced = ['combined_i.fits','aligned_to_i_z.fits','aligned_to_i_r.fits','aligned_to_i_g.fits']
curr_dir = ''
'''
for file in aligned_reduced:
    file_path = os.path.join(curr_dir, file)
    aperture_data = do_aperture_photometry(file_path, positions, radii, sky_radius_in, sky_annulus_width)
    mean_back = background_from_photometry(aperture_data, radii)
    output_path = os.path.join(curr_dir, f'bg_subtracted_{file}')
    subtract_background(file_path, mean_back, output_path)
'''
curr_dir = '/mnt/d/pixedfit_testing/ARCSAT/'
science_files = [
    'bg_subtracted_combined_i.fits',
    'bg_subtracted_aligned_to_i_z.fits',
    'bg_subtracted_aligned_to_i_r.fits',
    'bg_subtracted_aligned_to_i_g.fits'
]

for science_file in science_files:
    filter_name = 'i' if 'combined_i' in science_file else \
                  'z' if 'aligned_to_i_z' in science_file else \
                  'r' if 'aligned_to_i_r' in science_file else 'g'
    adu_files = sorted(glob.glob(os.path.join(curr_dir, f'new_reduced_science_{filter_name}*.fits')))
    with fits.open(adu_files[0]) as hdul:
        zmag = hdul[0].header['ZMAG']
        exptime = hdul[0].header['EXPTIME']
    science_path = os.path.join(curr_dir, science_file)
    create_error_image(science_path, adu_files, zmag, exptime, curr_dir)

Saved: /mnt/d/pixedfit_testing/ARCSAT/error_bg_subtracted_combined_i.fits, Sample: [1.1934070e-06 1.1897234e-06 1.1907816e-06 1.1933917e-06 1.2012332e-06]
Saved: /mnt/d/pixedfit_testing/ARCSAT/error_bg_subtracted_aligned_to_i_z.fits, Sample: [2.9392509e-06 2.9679113e-06 2.9440339e-06 2.9460655e-06 2.9613304e-06]
Saved: /mnt/d/pixedfit_testing/ARCSAT/error_bg_subtracted_aligned_to_i_r.fits, Sample: [9.972019e-07 9.919970e-07 9.789127e-07 9.815377e-07 9.878520e-07]
Saved: /mnt/d/pixedfit_testing/ARCSAT/error_bg_subtracted_aligned_to_i_g.fits, Sample: [1.5353215e-06 1.5201294e-06 1.5207438e-06 1.4990529e-06 1.5125976e-06]
